<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [75]:
df=pd.read_csv("NCELCash5.csv")
df

,Date,Number 1,Number 2,Number 3,Number 4,Number 5
0,9/25/20,1,5,28,37,42
1,9/24/20,2,20,27,35,40
2,9/23/20,9,15,26,33,37
3,9/22/20,1,4,8,18,35
4,9/21/20,1,4,6,29,35
...,...,...,...,...,...,...
5078,10/31/06,24,36,9,10,5
5079,10/30/06,18,2,32,14,25
5080,10/29/06,13,11,21,32,23
5081,10/28/06,12,2,18,28,39


In [24]:
Y=df.loc[:,["Number 1","Number 2","Number 3","Number 4","Number 5"]]

In [26]:
Y_train = keras.utils.to_categorical(Y["Number 1"], num_classes=43)   #turn it into on hot vector


In [27]:
Y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [28]:
query="jumanji"

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [72]:
import random as random
import time



In [122]:

def get(yyyy,mm,dd):
  URL="https://nclottery.com/Cash5Past"
  URL="https://nclottery.com/Cash5?dd="+str(mm)+"%2F"+str(dd)+"%2F"+str(yyyy)
  data={"ctl00_MainContent_ddlMonth":"1"
        ,"ctl00_MainContent_ddlYear":"2020"
        }

  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)

  #session.get(url)
  page = requests.get(URL)
  #page = session.post(URL, data=data)

  #page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #page = requests.get(URL)
  #results = soup#soup.find(id='ctl00_MainContent_lblCash5Match5Prize')
  results=soup.find(id="ctl00_MainContent_lblCash5TopPrize").text.strip()
  time.sleep(random.randint(0, 200)/100)
  return(results)

In [123]:
dates=[[d.year, d.month, d.day] for d in pd.date_range('2006-10-27', periods=5083)]

In [124]:
yyyy=dates[0][0]
mm=dates[0][1]
dd=dates[0][2]
print(dates[0])
get(yyyy,mm,dd)

[2006, 10, 27]


'$50,000'

In [125]:
cash=[get(yyyy,mm,dd) for yyyy,mm,dd in dates]

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [126]:
import concurrent.futures
import requests
import threading
import time

In [127]:
thread_local = threading.local()


In [183]:
def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

def get(yyyymmdd):
  yyyy,mm,dd=yyyymmdd
  #print(yyyy)
  URL="https://nclottery.com/Cash5?dd="+str(mm)+"%2F"+str(dd)+"%2F"+str(yyyy)
  session = get_session()

  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)

  #print(session)

  with session.get(URL) as page:
        #print(f"Read {len(page.content)} from {URL}")
        #page = requests.get(URL)
        #page = session.post(URL, data=data)
        #print(page)

        #page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        results=soup.find(id="ctl00_MainContent_lblCash5TopPrize").text.strip()
        #time.sleep(random.randint(0, 200)/100)
        #print(results)
        return([yyyy,mm,dd,results])

def get_all(yyyymmdd):
    a=[]
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        a=[i for i in executor.map(get, yyyymmdd)]
    return(a)

In [184]:
get_all([[2020,1,1],[2009,10,1]])

[[2020, 1, 1, '$134,000'], [2009, 10, 1, '$149,000']]

In [185]:
cash=get_all(dates)

In [189]:
pd.DataFrame(cash,columns=["yyyy","mm","dd","$$"]).to_csv("NCELCash5.winnings.csv")

In [167]:
get_all([[2020,1,1],[2009,10,1]])

<coroutine object get_all at 0x7fc828bfec50>

In [190]:
df1=pd.DataFrame(cash,columns=["yyyy","mm","dd","$$"])

In [194]:
df.Date

0        9/25/20
1        9/24/20
2        9/23/20
3        9/22/20
4        9/21/20
          ...   
5078    10/31/06
5079    10/30/06
5080    10/29/06
5081    10/28/06
5082    10/27/06
Name: Date, Length: 5083, dtype: object

In [198]:
from datetime import datetime
a="9/25/20"
date = datetime.strptime(a, '%m%d%Y')

ValueError: ignored

In [206]:
from dateutil import parser
parser.parse("6/9/01")

datetime.datetime(2001, 6, 9, 0, 0)

In [218]:
df["yyyy"]=[parser.parse(dt).year for dt in df.Date]
df["mm"]=[parser.parse(dt).month for dt in df.Date]
df["dd"]=[parser.parse(dt).day for dt in df.Date]


In [219]:
df

,Date,Number 1,Number 2,Number 3,Number 4,Number 5,Date1,yyyy,mm,dd
0,9/25/20,1,5,28,37,42,2020,2020,9,25
1,9/24/20,2,20,27,35,40,2020,2020,9,24
2,9/23/20,9,15,26,33,37,2020,2020,9,23
3,9/22/20,1,4,8,18,35,2020,2020,9,22
4,9/21/20,1,4,6,29,35,2020,2020,9,21
...,...,...,...,...,...,...,...,...,...,...
5078,10/31/06,24,36,9,10,5,2006,2006,10,31
5079,10/30/06,18,2,32,14,25,2006,2006,10,30
5080,10/29/06,13,11,21,32,23,2006,2006,10,29
5081,10/28/06,12,2,18,28,39,2006,2006,10,28


In [253]:
pd.DataFrame(
    keras.utils.to_categorical(Y["Number 1"], num_classes=44).astype(int)+
    keras.utils.to_categorical(Y["Number 2"], num_classes=44).astype(int)+
    keras.utils.to_categorical(Y["Number 3"], num_classes=44).astype(int)+
    keras.utils.to_categorical(Y["Number 4"], num_classes=44).astype(int)+
    keras.utils.to_categorical(Y["Number 5"], num_classes=44).astype(int)
    
)   #turn it into on hot vector


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5079,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5080,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5081,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [275]:
x=[
keras.utils.to_categorical(Y["Number 1"], num_classes=44).astype(int)+
keras.utils.to_categorical(Y["Number 2"], num_classes=44).astype(int)+
keras.utils.to_categorical(Y["Number 3"], num_classes=44).astype(int)+
keras.utils.to_categorical(Y["Number 4"], num_classes=44).astype(int)+
keras.utils.to_categorical(Y["Number 5"], num_classes=44).astype(int)
][0]
#''.join(x[0].tolist())
x

array([[0, 1, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [276]:
df["out"]=[''.join(map(str, y.tolist())) for y in x]

In [277]:
df

,Date,Number 1,Number 2,Number 3,Number 4,Number 5,Date1,yyyy,mm,dd,out
0,9/25/20,1,5,28,37,42,2020,2020,9,25,01000100000000000000000000001000000001000010
1,9/24/20,2,20,27,35,40,2020,2020,9,24,00100000000000000000100000010000000100001000
2,9/23/20,9,15,26,33,37,2020,2020,9,23,00000000010000010000000000100000010001000000
3,9/22/20,1,4,8,18,35,2020,2020,9,22,01001000100000000010000000000000000100000000
4,9/21/20,1,4,6,29,35,2020,2020,9,21,01001010000000000000000000000100000100000000
...,...,...,...,...,...,...,...,...,...,...,...
5078,10/31/06,24,36,9,10,5,2006,2006,10,31,00000100011000000000000010000000000010000000
5079,10/30/06,18,2,32,14,25,2006,2006,10,30,00100000000000100010000001000000100000000000
5080,10/29/06,13,11,21,32,23,2006,2006,10,29,00000000000101000000010100000000100000000000
5081,10/28/06,12,2,18,28,39,2006,2006,10,28,00100000000010000010000000001000000000010000


In [283]:
df2=df.copy()

In [284]:
df2.drop(columns=["Date","Date1","Number 1","Number 2","Number 3","Number 4","Number 5"], inplace=True)

In [285]:
df2

,yyyy,mm,dd,out
0,2020,9,25,01000100000000000000000000001000000001000010
1,2020,9,24,00100000000000000000100000010000000100001000
2,2020,9,23,00000000010000010000000000100000010001000000
3,2020,9,22,01001000100000000010000000000000000100000000
4,2020,9,21,01001010000000000000000000000100000100000000
...,...,...,...,...
5078,2006,10,31,00000100011000000000000010000000000010000000
5079,2006,10,30,00100000000000100010000001000000100000000000
5080,2006,10,29,00000000000101000000010100000000100000000000
5081,2006,10,28,00100000000010000010000000001000000000010000


In [288]:
df1

,yyyy,mm,dd,$$
0,2006,10,27,"$50,000"
1,2006,10,28,"$75,000"
2,2006,10,29,"$100,000"
3,2006,10,30,"$50,000"
4,2006,10,31,"$60,000"
...,...,...,...,...
5078,2020,9,21,"$644,000"
5079,2020,9,22,"$794,000"
5080,2020,9,23,"$1,000,000"
5081,2020,9,24,"$1,200,000"


In [290]:
df3=df2.merge(df1,on=["yyyy","mm","dd"])

In [299]:
df3["$$"]=[ int(str(i).translate(str.maketrans({'$': '', ',': ''}))) for i in df3["$$"]]
      

In [301]:
df3=df3[["yyyy","mm","dd","$$","out"]]

In [302]:
df3

,yyyy,mm,dd,$$,out
0,2020,9,25,100000,01000100000000000000000000001000000001000010
1,2020,9,24,1200000,00100000000000000000100000010000000100001000
2,2020,9,23,1000000,00000000010000010000000000100000010001000000
3,2020,9,22,794000,01001000100000000010000000000000000100000000
4,2020,9,21,644000,01001010000000000000000000000100000100000000
...,...,...,...,...,...
5078,2006,10,31,60000,00000100011000000000000010000000000010000000
5079,2006,10,30,50000,00100000000000100010000001000000100000000000
5080,2006,10,29,100000,00000000000101000000010100000000100000000000
5081,2006,10,28,75000,00100000000010000010000000001000000000010000


In [303]:
df3.to_csv("NCELCash5.winnings.1he.dollar.csv", index=False)